In [62]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

import pandas as pd


In [3]:
engine = create_engine("sqlite:///datasets/belly_button_biodiversity.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
otu = Base.classes.otu
samples = Base.classes.samples
samples_meta = Base.classes.samples_metadata

# Create our session (link) from Python to the DB
session = Session(engine)


In [5]:
inspector = inspect(engine)

In [6]:
inspector.get_table_names()

['otu', 'samples', 'samples_metadata']

In [7]:
columns = inspector.get_columns('otu')
for column in columns:
    print(column["name"], column["type"])

otu_id INTEGER
lowest_taxonomic_unit_found TEXT


In [44]:
columns = inspector.get_columns('samples_metadata')
for column in columns:
    print(column["name"], column["type"])

SAMPLEID INTEGER
EVENT TEXT
ETHNICITY TEXT
GENDER TEXT
AGE INTEGER
WFREQ INTEGER
BBTYPE TEXT
LOCATION TEXT
COUNTRY012 TEXT
ZIP012 INTEGER
COUNTRY1319 TEXT
ZIP1319 INTEGER
DOG TEXT
CAT TEXT
IMPSURFACE013 INTEGER
NPP013 FLOAT
MMAXTEMP013 FLOAT
PFC013 FLOAT
IMPSURFACE1319 INTEGER
NPP1319 FLOAT
MMAXTEMP1319 FLOAT
PFC1319 FLOAT


In [27]:
#app.route('/names')
def sample_names():
    columns = inspector.get_columns('samples')
    names = []
    for column in columns:
        names.append(column['name'])
    del names[0]
    
    return jsonify(names)

In [133]:
#app.route('/otu')
def otu_descriptions():
    results = session.query(otu.otu_id, otu.lowest_taxonomic_unit_found).all()
    df = pd.DataFrame(results)

    otu_count = len(df.index)
    otu_info = []
    for x in range(otu_count):
        otu_info.append({df['otu_id'][x]: df['lowest_taxonomic_unit_found'][x]})
        
    return jsonify(otu_info)

[{1: 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'},
 {2: 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'},
 {3: 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Natronorubrum'},
 {4: 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'},
 {5: 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'},
 {6: 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'},
 {7: 'Bacteria'},
 {8: 'Bacteria'},
 {9: 'Bacteria'},
 {10: 'Bacteria'},
 {11: 'Bacteria'},
 {12: 'Bacteria'},
 {13: 'Bacteria'},
 {14: 'Bacteria'},
 {15: 'Bacteria'},
 {16: 'Bacteria'},
 {17: 'Bacteria'},
 {18: 'Bacteria'},
 {19: 'Bacteria'},
 {20: 'Bacteria'},
 {21: 'Bacteria'},
 {22: 'Bacteria'},
 {23: 'Bacteria'},
 {24: 'Bacteria'},
 {25: 'Bacteria'},
 {26: 'Bacteria'},
 {27: 'Bacteria'},
 {28: 'Bacteria'},
 {2

In [ ]:
#@app.route('/metadata/<sample>')
def sample_metadata(sample):
    sample_id = sample.replace('BB_','')
    result = session.query(samples_meta).filter(samples_meta.SAMPLEID == sample_id).first()
    metadata = {
        'AGE':result.AGE,
        'BBTYPE':result.BBTYPE,
        'ETHNICITY':result.ETHNICITY,
        'GENDER':result.GENDER,
        'LOCATION':result.LOCATION,
        'SAMPLEID':result.SAMPLEID
    }
    
    return jsonify(metadata)

In [ ]:
#app.route('/wfreq/<sample>')
def washing_frequency(sample):
    sample_id = sample.replace('BB_','')
    result = session.query(samples_meta).filter(samples_meta.SAMPLEID == sample_id).first()
    
    return result.WFREQ

In [ ]:
#app.route('/samples/<sample>')
def sample_count(sample):
    results = session.query(samples.otu_id, getattr(samples, sample)).all()
    df = pd.DataFrame(results)
    df = df.set_index('otu_id').sort_values(by=[sample], ascending=False).head(10).reset_index()
    ids = tuple(df['otu_id'].values.tolist())
    vals = tuple(df[sample].values.tolist())
    sample_counts = {'otu_ids':ids,'sample_values':vals}
    
    return jsonify(sample_counts)